#CafChem tools for using a Open Targets to find targets for a disease

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MauricioCafiero/CafChem/blob/main/notebooks/OpenTargets_CafChem.ipynb)

## This notebook allows you to:
- Search for protein targets from a disease name by querying Open Targets

## Requirements:
- CPU runtime is fine!

## Imports

In [1]:
import requests
import json

## Define the searching function

In [7]:
def target_node(search_descriptor: str):
  '''
  Accepts a disease name and searches Open Targets for associated targets

  Args:
    search_descriptor (str): Disease name

  Returns:
    targets_list (list): List of targets
    targets_string (str): String of targets
    None
  '''
  base_url = "https://api.platform.opentargets.org/api/v4/graphql"

  disease_query_string = """
    query searchEntity($queryString: String!) {
      search(queryString: $queryString){
        total
        hits  {
          id
          entity
          description
        }
      }
    }
  """

  target_query_string = """
    query associatedTargets($efo_id: String!) {
      disease(efoId: $efo_id) {
        id
        name
        associatedTargets {
          count
          rows {
            target {
              id
              approvedSymbol
            }
            score
          }
        }
      }
    }
  """

  variables = {"queryString": search_descriptor}
  r = requests.post(base_url, json={"query": disease_query_string, "variables": variables})

  disease_list = []
  targets_list = []

  if r.status_code == 200:
    api_response = json.loads(r.text)
    if len(api_response['data']['search']['hits']) > 0:
      for hit in api_response['data']['search']['hits']:
        if hit['entity'] == 'disease':
          disease_list.append(hit['id'])
  else:
    print('Could not find results.')

  if len(disease_list) > 0:
    q = requests.post(base_url, json={"query": target_query_string, "variables": {"efo_id": disease_list[0]}})
    if q.status_code == 200:
      api_response = json.loads(q.text)
      for target in api_response['data']['disease']['associatedTargets']['rows']:
        targets_list.append(target['target']['approvedSymbol'])

  targets_string = f'Possible targets for {search_descriptor} include: \n'
  if len(targets_list) > 0:
    for i, target in enumerate(targets_list):
      targets_string += f'{i+1}. {target}\n'
  else:
    targets_string = f'No targets found for {search_descriptor}'

  return targets_list, targets_string, None

## Sample searches

In [5]:
result_list, result_string, _ = target_node("Parkinsons disease")
print(result_string)

No targets found for Parkinsons disease


In [6]:
result_list, result_string, _ = target_node("Phenylketonuria")
print(result_string)

Possible targets for Phenylketonuria include: 
1. PAH
2. DNAJC12
3. NSUN2
4. COL1A1
5. PI4KA
6. POMGNT1
7. RABEP1
8. ADAMTS6
9. SLC6A19
10. MEGF8
11. IFT25
12. HCRT
13. FOXE3
14. MYO10
15. GRIN2B
16. ATP1A3
17. SLC7A5
18. HNF1A
19. MECP2
20. OTC
21. TBC1D32
22. PDGFRB
23. ACADM
24. FOXH1
25. CPLANE1

